<a href="https://colab.research.google.com/github/tftr0141/AIart/blob/main/imagic/Imagic_Stable_Diffusion_Lisa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 概要

Imagic Stable Diffusion を動かすためのノートブックです

画像の一部だけをうまい具合に加工してくれます

* img2imgやinpaintingとの比較
  * メリット
    * Imagicの方が元の画像の構図や特徴を活かしてくれます
    * 上手くやるとポーズの変更など、大きな可能もできそうです
  * デメリット
    * 画像変換の準備（学習データの用意）に時間がかかります
      * 標準設定で20分～30分程度
    * パラメータの調整が現状は手作業なので手間がかかります


* 参考
  * Imagic Stable Diffusionの解説がわかりやすい記事
    * https://birdmanikioishota.blog.fc2.com/blog-entry-12.html
  * このノートブックを作るのに参考にしたノートブック
    * https://colab.research.google.com/github/ShivamShrirao/diffusers/blob/main/examples/imagic/Imagic_Stable_Diffusion.ipynb
  * 元レポジトリなど
    * https://github.com/ShivamShrirao/diffusers/tree/main/examples/imagic
    

In [ ]:
#@markdown ## 1. 必要なパッケージのインストール(Install Requirements)
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/imagic/train_imagic.py
#%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -qq git+https://github.com/ShivamShrirao/diffusers@d34dcaa3c1913c7b794f8438c7c3c5ded265fa48
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio


#Install xformers from precompiled wheel.
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

# If precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

In [ ]:
#@title 2.HuggingFaceへのログイン（Login to HuggingFace 🤗）
#@markdown Stable-Diffusionを以外のモデルを使う場合でも必要です
#@markdown 
#@markdown WaifuDiffusionなど別モデルを使う場合は不要です
#You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

In [ ]:
#@markdown ## 3.設定
MODEL_NAME = "Nilaier/Waifu-Diffusers" #@param ["hakurei/waifu-diffusion", "Nilaier/Waifu-Diffusers","CompVis/stable-diffusion-v1-4", "naclbit/trinart_stable_diffusion_v2,diffusers-115k", "naclbit/trinart_stable_diffusion_v2,diffusers-95k", "naclbit/trinart_stable_diffusion_v2,diffusers-60k"] {allow-input: true}
#@markdown 利用するモデルを選択
#@markdown   * CompVis/stable-diffusion-v1-4 → 素のStableDiffusion
#@markdown   * hakurei/waifu-diffusion → Waifu-Diffusion 1.2
#@markdown   * Nilaier/Waifu-Diffusers → Waifu-Diffusion 1.3
#@markdown ----
TARGET_TEXT = "white background, girl wearing glasses" #@param {type:"string"}
#@markdown 変更したい絵を書いて下さい
#@markdown * 例
#@markdown   * 写真の犬を走らせたい→a dog running
#@markdown   * 女の子にメガネをかけさせたい→girl wearing glasses
#@markdown     * beautifulとかkawaiiとか装飾語をつけたほうが良いのかは未調査だけど、多分つけたほうが良さそう
#@markdown ----

emb_train_steps = 500 #@param {type:"number"}
max_train_steps = 1000 #@param {type:"number"}
#@markdown * 学習用パラメータ。ツールのデフォルト値は500と1000
#@markdown * 増やすと精度が高くなる分、時間がかかります
#@markdown   * メガネ追加の検証時は、1000・2000にしたほうが安定したので、時間に余裕があるなら増やした方安全かもです
#@markdown   * 500/1000で学習に20分、1000/2000で40分程度かかります
#@markdown ----
#@markdown * 学習用パラメータ。デフォルトは1000
#@markdown ----

IMG_OUTPUT_PATH = "/content/drive/MyDrive/stable_diffusion_weights/results" #@param {type:"string"}
#@markdown * 結果画像の保存先
#@markdown ----

save_to_gdrive = False #@param {type:"boolean"}
#@markdown * GoogleDriveにモデルデータを保存させたい場合は、チェックを付けて下さい（基本は無しでOK）
#@markdown ----

from google.colab import drive
drive.mount('/content/drive')

OUTPUT_DIR = "stable_diffusion_weights/imagic" #@param {type:"string"}
#@markdown * モデルデータの保存先（基本はそのままでOK）
#@markdown ----
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")
!mkdir -p $OUTPUT_DIR
!mkdir -p $IMG_OUTPUT_PATH


In [ ]:
#@markdown ## 4.元画像をアップロード
#@markdown 512x512以外で動くかは確認してません

import os
from google.colab import files
import shutil

uploaded = files.upload()
for filename in uploaded.keys():
    INPUT_IMAGE = os.path.join(OUTPUT_DIR, filename)
    shutil.move(filename, INPUT_IMAGE)

In [ ]:
#@markdown ## 5.学習実行
#@markdown 標準設定で20～30分程度かかります
!accelerate launch train_imagic.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --output_dir=$OUTPUT_DIR \
  --input_image=$INPUT_IMAGE \
  --target_text="{TARGET_TEXT}" \
  --seed=3434554 \
  --resolution=512 \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --emb_learning_rate=1e-3 \
  --learning_rate=1e-6 \
  --emb_train_steps={emb_train_steps} \
  --max_train_steps={max_train_steps}


In [ ]:
#@markdown ## （任意）ckptファイルへ変換（AUTOMATIC1111などで利用する場合）
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
ckpt_path = OUTPUT_DIR + "/model.ckpt"

half_arg = ""
fp16 = True 
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $OUTPUT_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

In [ ]:
#@markdown ## 6. 画像出力準備
#@markdown 30秒程度で終わります
import os
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = OUTPUT_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, torch_dtype=torch.float16).to("cuda")
target_embeddings = torch.load(os.path.join(model_path, "target_embeddings.pt")).to("cuda")
optimized_embeddings = torch.load(os.path.join(model_path, "optimized_embeddings.pt")).to("cuda")
g_cuda = torch.Generator(device='cuda')

In [ ]:
#@markdown ## 7.画像出力UI起動
#@markdown 出力結果は GoogleDrive の stable_diffusion_weights/results にも保存されます
#@markdown ### 期待する画像を見つけるためのパラメータ調整
#@markdown 運が良いと一度で目当ての画像を引ける場合もありますが、パラメータを調整しないと出ない場合も多いです
#@markdown 
#@markdown * 画像が元の画像のままの場合
#@markdown   * promptの値を変えて実行し、元の画像と変換後の画像の境目を探しましょう
#@markdown   * 境目だと、0.1違うだけで出力結果が大きく違ったりします
#@markdown   * こだわるなら0.01単位でこだわっても良さそうなレベル感でした
#@markdown   * 最適値は画像やテキストによって大きく異なる感じですが、最適値は0.5～1.5の間ぐらいな気がしてます
#@markdown * 画像がテキスト寄りでぼやけてる
#@markdown   * guidance_scaleを増やすとハッキリした絵になるかもしれないです
#@markdown   * guidance_scaleを増やしすぎると、原型が無くなるので、程々を探してみて下さい
#@markdown   * 最適値は条件によって大きく異なり、3でOKの場合もあれば50近くまで増やして安定したケースもありました
#@markdown * guidance_scaleを調整してもぼやける場合
#@markdown   * stepsを増やすと良くなるケースもありますが、その分描画時間が長くなります
#@markdown   * stepsを増やすのは最後の仕上げぐらいが良い気がします
#@markdown * それでも上手くいかない場合
#@markdown   * 学習ステップ数が足りない可能性があります
#@markdown   * emb_train_stepsやmax_train_stepsを倍の値にして上手くいったケースもあるので、学習パラメータを増やして最初からやり直しましょう
#@markdown * その他パラメータ
#@markdown   * Number of Samplesはあまり増やすとGPUのメモリ不足になるので、扱い注意
#@markdown   * wiidth/heightは変えると画像が崩壊しがちなので、これも取り扱い注意です


import gradio as gr
import datetime
def getTimeText():
    t_delta = datetime.timedelta(hours=9)
    JST = datetime.timezone(t_delta, 'JST')
    now = datetime.datetime.now(JST)
    return now.strftime('%Y%m%d_%H%M%S')

def inference(alpha, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5, seed = 100):
    g_cuda.manual_seed(int(seed))
    with torch.autocast("cuda"), torch.inference_mode():
        edit_embeddings = alpha*target_embeddings + (1-alpha)*optimized_embeddings
        images = pipe(
                text_embeddings=edit_embeddings, height=int(height), width=int(width),
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images
        now = getTimeText()
        i = 0
        for image in images:
          i = i + 1
          save_name = f"{now}_{i}_{alpha}_{seed}_{num_samples}_{guidance_scale}_{num_inference_steps}.png"
          image.save(os.path.join(IMG_OUTPUT_PATH,save_name))
        return images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            with gr.Row():
              alpha = gr.Number(label="Prompt", value=0.9)
              seed = gr.Number(label="Seed", value=100)
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=3)
            num_inference_steps = gr.Slider(label="Steps", value=20, maximum=200)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[alpha, num_samples, height, width, num_inference_steps, guidance_scale,seed], outputs=gallery)

demo.launch(debug=True)

In [ ]:
#@markdown ## 8.セッションの切断
#@markdown セッションを繋いだままだとクレジットを消費するので、実行が終わったら切断しましょう
from google.colab import runtime
runtime.unassign()